# LQR Control in pykal

This notebook demonstrates the implementation of a Linear Quadratic Regulator (LQR) controller in pykal, based on Kalman's seminal work on optimal control theory [1].

## Introduction

The LQR controller computes the optimal state feedback gain that minimizes a quadratic cost function for linear discrete-time systems. It is widely used in aerospace, robotics, and process control applications.

**Problem formulation:**

For a linear discrete-time system:

$$x_{k+1} = A x_k + B u_k$$

The LQR controller minimizes the infinite-horizon cost:

$$J = \sum_{k=0}^{\infty} \left( x_k^T Q x_k + u_k^T R u_k \right)$$

where:
- $Q$ (n × n) penalizes state deviations (positive semi-definite)
- $R$ (m × m) penalizes control effort (positive definite)

**Optimal control law:**

$$u_k = -K (x_k - x_{\text{ref}})$$

where $K$ is the optimal feedback gain computed by solving the Discrete-Time Algebraic Riccati Equation (DARE).

## References

[1] Kalman, R. E. (1960). Contributions to the theory of optimal control. *Boletin de la Sociedad Matematica Mexicana*, 5(2), 102-119.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.algorithm_library.controllers.lqr import LQR

## System Model: Double Integrator

We'll control a double integrator system (e.g., a mass on a frictionless surface):

$$\ddot{x} = u$$

where $x$ is position and $u$ is acceleration (control input).

**State-space representation:**

State: $\mathbf{x} = [x, \dot{x}]^T$ (position, velocity)

**Continuous-time:**
$$\dot{\mathbf{x}} = \begin{bmatrix} 0 & 1 \\ 0 & 0 \end{bmatrix} \mathbf{x} + \begin{bmatrix} 0 \\ 1 \end{bmatrix} u$$

**Discrete-time (Euler integration, dt=0.1):**
$$\mathbf{x}_{k+1} = \begin{bmatrix} 1 & 0.1 \\ 0 & 1 \end{bmatrix} \mathbf{x}_k + \begin{bmatrix} 0.005 \\ 0.1 \end{bmatrix} u_k$$

In [ ]:
# Simulation parameters
dt = 0.1  # 100 ms time step
T = 10.0  # 10 second simulation
N = int(T / dt)
time = np.arange(N) * dt

# Discrete-time system matrices (Euler discretization)
A = np.array([[1.0, dt],
              [0.0, 1.0]])
B = np.array([[0.5 * dt**2],
              [dt]])

print(f"System matrices:")
print(f"A = \n{A}")
print(f"B = \n{B}")

# Check controllability
C_matrix = np.hstack([B, A @ B])
rank = np.linalg.matrix_rank(C_matrix)
print(f"\nControllability matrix rank: {rank} (system dimension: {A.shape[0]})")
print(f"System is {'controllable' if rank == A.shape[0] else 'NOT controllable'}")

## LQR Gain Computation

We'll design an LQR controller with the following tuning:

- $Q = \text{diag}(10, 1)$: Prioritize position error (10×) over velocity error
- $R = 1$: Moderate control effort penalty

This balance will give us fast position tracking with reasonable control inputs.

In [ ]:
# LQR cost matrices
Q = np.diag([10.0, 1.0])  # State cost: prioritize position
R = np.array([[1.0]])      # Control cost

# Compute optimal LQR gain
K, P = LQR.compute_gain(A, B, Q, R)

print(f"Optimal feedback gain K:")
print(K)
print(f"\nDARE solution P:")
print(P)

# Analyze closed-loop stability
A_cl = A - B @ K
eigenvalues = np.linalg.eigvals(A_cl)
print(f"\nClosed-loop eigenvalues: {eigenvalues}")
print(f"Spectral radius: {np.max(np.abs(eigenvalues)):.4f}")
print(f"System is {'stable' if np.all(np.abs(eigenvalues) < 1.0) else 'UNSTABLE'}")

## Create DynamicalSystem Instances

We'll create:
1. **Plant**: The double integrator system
2. **Controller**: The LQR controller

In [ ]:
# Plant dynamics
def plant_f(x, u):
    """Double integrator state evolution."""
    return A @ x + B.flatten() * u

def plant_h(x):
    """Full state observation."""
    return x

# Initial conditions
x0 = np.array([5.0, 0.0])  # Start 5 units from origin, at rest
lqr_state0 = (K, P)        # LQR controller state

# Shared parameter dictionary
params = {
    'x': x0,
    'lqr_state': lqr_state0,
    'u': 0.0,
    'xhat_k': x0,
    'xref_k': np.array([0.0, 0.0])  # Regulate to origin
}

# Create plant system
plant = DynamicalSystem(
    f=plant_f,
    h=plant_h,
    state_name='x'
)

# Create LQR controller
controller = DynamicalSystem(
    f=LQR.f,
    h=LQR.h,
    state_name='lqr_state'
)

print("DynamicalSystem instances created successfully!")

## Run Closed-Loop Simulation

We'll simulate the closed-loop system tracking a step reference that changes at t=5s.

In [ ]:
# Storage arrays
x_history = np.zeros((N, 2))
u_history = np.zeros(N)
r_history = np.zeros((N, 2))

# Simulation loop
for k in range(N):
    # Update reference (step change at t=5s)
    if time[k] < 5.0:
        params['xref_k'] = np.array([0.0, 0.0])  # Origin
    else:
        params['xref_k'] = np.array([3.0, 0.0])  # Move to x=3
    
    # Get current state (measurement)
    x_k = plant.step(param_dict=params)
    params['xhat_k'] = x_k  # Perfect state feedback
    
    # Controller computes control input
    u_k = controller.step(param_dict=params)
    params['u'] = u_k[0] if isinstance(u_k, np.ndarray) else u_k
    
    # Store history
    x_history[k] = x_k
    u_history[k] = params['u']
    r_history[k] = params['xref_k']

print("Simulation complete!")

## Results Visualization

Let's visualize the tracking performance, control effort, and phase portrait.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 9))

# Plot 1: Position tracking
axes[0].plot(time, r_history[:, 0], 'r--', linewidth=2, label='Reference $x_{ref}(t)$')
axes[0].plot(time, x_history[:, 0], 'b-', linewidth=1.5, label='Actual $x(t)$')
axes[0].set_ylabel('Position', fontsize=12)
axes[0].set_title('LQR Control Performance: Double Integrator', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Velocity
axes[1].plot(time, r_history[:, 1], 'r--', linewidth=2, label='Reference $\dot{x}_{ref}(t)$')
axes[1].plot(time, x_history[:, 1], 'b-', linewidth=1.5, label='Actual $\dot{x}(t)$')
axes[1].set_ylabel('Velocity', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# Plot 3: Control input
axes[2].plot(time, u_history, 'g-', linewidth=1.5)
axes[2].set_ylabel('Control Input $u(t)$', fontsize=12)
axes[2].set_xlabel('Time (seconds)', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Phase portrait
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(x_history[:, 0], x_history[:, 1], 'b-', linewidth=1.5, label='Trajectory')
ax.plot(x_history[0, 0], x_history[0, 1], 'go', markersize=10, label='Start')
ax.plot(x_history[-1, 0], x_history[-1, 1], 'rs', markersize=10, label='End')
ax.plot(r_history[:, 0], r_history[:, 1], 'r*', markersize=12, label='References')

ax.set_xlabel('Position $x$', fontsize=12)
ax.set_ylabel('Velocity $\dot{x}$', fontsize=12)
ax.set_title('Phase Portrait', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Performance Metrics

In [ ]:
# Compute performance metrics
position_error = x_history[:, 0] - r_history[:, 0]
velocity_error = x_history[:, 1] - r_history[:, 1]

# Settling time (2% criterion)
settling_threshold = 0.02 * np.abs(x0[0])
settled_indices = np.where(np.abs(position_error[:50]) < settling_threshold)[0]
settling_time = time[settled_indices[-1]] if len(settled_indices) > 0 else None

print("\nPerformance Metrics:")
print(f"Final position error: {position_error[-1]:.6f}")
print(f"Final velocity error: {velocity_error[-1]:.6f}")
print(f"RMS position error: {np.sqrt(np.mean(position_error**2)):.6f}")
print(f"Max control effort: {np.max(np.abs(u_history)):.3f}")
if settling_time:
    print(f"Settling time (2%): {settling_time:.2f} s")

# Verify cost reduction
state_cost = np.sum([x.T @ Q @ x for x in x_history])
control_cost = np.sum([u**2 * R[0,0] for u in u_history])
total_cost = state_cost + control_cost
print(f"\nTotal cost J: {total_cost:.2f}")
print(f"  State cost:   {state_cost:.2f}")
print(f"  Control cost: {control_cost:.2f}")

## Conclusion

This notebook demonstrated:

✓ **LQR controller implementation** in pykal using the `DynamicalSystem` framework

✓ **DARE solution** via `LQR.compute_gain()` for optimal feedback gain

✓ **Closed-loop simulation** of a double integrator system

✓ **Performance analysis** showing fast convergence and optimality

The LQR controller successfully:
- Stabilized the double integrator from initial condition $x_0 = [5, 0]$ to origin
- Tracked a step reference change at $t = 5$ s
- Minimized the quadratic cost function
- Guaranteed closed-loop stability (eigenvalues inside unit circle)

### Next Steps:

- **Deploy on TurtleBot3**: See `lqr_turtlebot.ipynb` for trajectory tracking on a mobile robot
- **Deploy on Crazyflie**: See `lqr_crazyflie.ipynb` for attitude control on a quadrotor
- **Experiment with tuning**: Adjust $Q$ and $R$ to trade off performance vs. control effort
- **Add disturbances**: Test robustness to external forces and measurement noise